In [4]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=25a5166da8bbec7bee143878ce4d179fcd56a86504b81f7bbcdd130ab4e87210
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("COMP5349 Aassignmen1").getOrCreate()

In [6]:
import string
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
punctuation = string.punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Method 1

## Governing_law


###  1-Map the original RDD to the tuple like [(filename1, clause1),(filename2, clause1)]

In [7]:
# The file is in the comp5349 working directory.
Governing_law = spark.read.csv('/content/drive/MyDrive/comp5349/Governing_Law.csv', header = True).rdd
Governing_law.take(5)

[Row(Filename='CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf', Governing Law='This Agreement is accepted by Company in the State of Nevada and shall be governed by and construed in accordance with the laws thereof, which laws shall prevail in the event of any conflict. (Page 13)'),
 Row(Filename='EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf', Governing Law='This Agreement shall be governed by laws of the Province of Ontario and the federal laws of Canada applicable therein. (Page 9)  This Agreement is subject to all laws, regulations, license conditions and decisions of the Canadian Radio-television and Telecommunications Commission (‚ÄúCRTC‚Äù) municipal, provincial and federal governments or other authorities which are applicable to Rogers and/or Licensor, and which are now in force or hereafter adopted (‚ÄúApplicable Law‚Äù). (Page 11)'),
 Row(Filename='FulucaiProductionsLtd_20131223_10-Q_EX-

In [8]:
# Firstly, map to this format(filename, clause)
def row_to_tuple (row):
  row_dict = row.asDict()
  return (row_dict["Filename"], row_dict["Governing Law"])

gov_rdd = Governing_law.map(row_to_tuple)

In [9]:
# Secondly, map to this format [(filename1, clause1), (filename2, clause2)], cause one file many contains more than one clause.
def method1_extract(record):
  filename, docs = record[0], record[1]
  doc = docs.lower() # convert lowercase
  doc = re.sub(r'\([Pp]age.*?\)', "|", doc) 
#  doc = re.sub('\([pP]age \w*\)', "|", doc) 
  sentence = doc.split("|")
  ls = [] 
  i = 1             
  for sc in sentence:
    if sc != '':               # remove empty value
      filename_num = filename +  str(i)  
      ls.append((filename_num, sc))
      i += 1
  return ls

#  filter nan values and  spaces
method1_gov_rdd1 = gov_rdd.filter(lambda x: x[1] != 'nan').flatMap(method1_extract).map(lambda x: (x[0],x[1].strip())) 
method1_gov_rdd1.take(5)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  'this agreement is accepted by company in the state of nevada and shall be governed by and construed in accordance with the laws thereof, which laws shall prevail in the event of any conflict.'),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf1',
  'this agreement shall be governed by laws of the province of ontario and the federal laws of canada applicable therein.'),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  'this agreement is subject to all laws, regulations, license conditions and decisions of the canadian radio-television and telecommunications commission (‚äúcrtc‚äù) municipal, provincial and federal governments or other authorities which are applicable to rogers and/or licensor, and which are now in force or hereafter adopted (‚äúapplicable law‚äù).'),
 ('FulucaiProducti

### 2-candidate Phrase Identification

removing stopwords and punctuation, then generating candidate phrases.

In [10]:
# map to the format like (filename, list of candidate phrases)

def method1_preprocessing(record):
  docs = ''
  for word in record[1]:
    if word in punctuation:     # using "|" to replace the punctuation
      docs += "|"
    else:
      docs += word

  docs = re.sub(r'[^\w\s]','', docs)  # remove other punctuations which cannot be identified by the list of punctuation defined in the previous cell.

  doc_word = docs.split(" ")
  for word in stopwords:
      doc_word = [ "|" if word == w else w for w in doc_word]  # using "|" to replace the stopwords
     
 
  sentence = " ".join(doc_word)  
  sentence = sentence.split("|")  
  return (record[0],sentence)


method1_gov_rdd2 = method1_gov_rdd1.map(method1_preprocessing)
method1_gov_rdd2.take(1)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  ['',
   ' agreement ',
   ' accepted ',
   ' company ',
   ' ',
   ' state ',
   ' nevada ',
   ' shall ',
   ' governed ',
   ' ',
   ' construed ',
   ' accordance ',
   ' ',
   ' laws thereof ',
   ' laws shall prevail ',
   ' ',
   ' event ',
   ' ',
   ' conflict'])]

In [11]:
# remove extra spaces
# Map the candidate phrases to the list format, and each word is split by comma for each candidate phrase of each document
def remove_space_list(record):
  sentence = [word.strip() for word in record[1]]
  ls = []
  for i in sentence:
    if i != '':
      ls.append(i)

  ls = [ word.split() for word in ls]
  return (record[0], ls)

method1_gov_rdd_list = method1_gov_rdd2.map(remove_space_list)
method1_gov_rdd_list.take(1)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  [['agreement'],
   ['accepted'],
   ['company'],
   ['state'],
   ['nevada'],
   ['shall'],
   ['governed'],
   ['construed'],
   ['accordance'],
   ['laws', 'thereof'],
   ['laws', 'shall', 'prevail'],
   ['event'],
   ['conflict']])]

### 3- Word Score Calculation

In [12]:
# Generate the tuple like ((filename, single_word), candidate phrase,  fre(w), deg(w))) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
def ls_fre_deg(record):
  res = []
  for word_ls in record[1]:
    for word in word_ls:
      res.append(((record[0], word), word_ls, 1, len(word_ls)))
  
  return res

method1_gov_rdd2 = method1_gov_rdd_list.flatMap(ls_fre_deg)
method1_gov_rdd2.take(3)

[(('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'agreement'),
  ['agreement'],
  1,
  1),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'accepted'),
  ['accepted'],
  1,
  1),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'company'),
  ['company'],
  1,
  1)]

In [13]:
# This RDD is for joining later
method1_gov_rdd = method1_gov_rdd2.map(lambda x: (x[0], x[1]))
method1_gov_rdd.take(3)

[(('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'agreement'),
  ['agreement']),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'accepted'),
  ['accepted']),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'company'),
  ['company'])]

#### 3.1.Fre(w) calculation for each word

In [14]:
method1_gov_fre = method1_gov_rdd2.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_gov_fre.take(5)

[(('EdietsComInc_20001030_10QSB_EX-10.4_2606646_EX-10.4_Co-Branding Agreement.pdf1',
   'california'),
  5),
 (('INTERNATIONALFASTFOODCORP_04_04_1997-EX-99-FRANCHISE AGREEMENT.PDF1',
   'florida'),
  5),
 (('SightLife Surgical, Inc. - STRATEGIC SALES & MARKETING AGREEMENT.PDF1',
   'state'),
  5),
 (('HealthcareIntegratedTechnologiesInc_20190812_8-K_EX-10.1_11776966_EX-10.1_Reseller Agreement.pdf1',
   'shall'),
  4),
 (('ElPolloLocoHoldingsInc_20200306_10-K_EX-10.16_12041700_EX-10.16_Development Agreement.pdf1',
   'state'),
  4)]

#### 3.2.Deg(w) calculation for each word

In [15]:
method1_gov_deg = method1_gov_rdd2.map(lambda x: (x[0],  x[3])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_gov_deg.take(10)

[(('INTERNATIONALFASTFOODCORP_04_04_1997-EX-99-FRANCHISE AGREEMENT.PDF1',
   'florida'),
  23),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF1', 'act'), 23),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF1', 'usc'), 19),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF1', 'et'), 19),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF1', 'seq'), 19),
 (('HealthcareIntegratedTechnologiesInc_20190812_8-K_EX-10.1_11776966_EX-10.1_Reseller Agreement.pdf1',
   'new'),
  16),
 (('HealthcareIntegratedTechnologiesInc_20190812_8-K_EX-10.1_11776966_EX-10.1_Reseller Agreement.pdf1',
   'york'),
  16),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement2.pdf1',
   'new'),
  16),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement2.pdf1',
   'york'),
  16),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement1.pdf1',
   'new'),
  16)]

#### 3.3.Ratio of fre(w) and deg(w) calculation for each word

In [16]:
method1_gov_score = method1_gov_fre.join(method1_gov_deg).map(lambda x: (x[0],x[1][1]/x[1][0]))
method1_gov_score.take(5)

[(('HealthcareIntegratedTechnologiesInc_20190812_8-K_EX-10.1_11776966_EX-10.1_Reseller Agreement.pdf1',
   'shall'),
  2.25),
 (('ElPolloLocoHoldingsInc_20200306_10-K_EX-10.16_12041700_EX-10.16_Development Agreement.pdf1',
   'state'),
  1.25),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement1.pdf1',
   'new'),
  4.0),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement1.pdf1',
   'york'),
  4.0),
 (('AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement1.pdf1',
   'agreement'),
  1.0)]

### 4-Candidate Phrase Score Calculation

In [17]:
# join the RDD lile (word,candidate phrase) and ((word, score) RDD
# using map and reduceByKey operations to generate like the following format (filename, (candidate phrase, score) )
method1_gov_rdd4 = method1_gov_rdd.join(method1_gov_score).map(lambda x: (x[0][0],(tuple(x[1][0]),x[1][1])))
method1_gov_rdd5 = method1_gov_rdd4.map(lambda x: ((x[0], x[1][0]),x[1][1])).reduceByKey(lambda a,b: a+b)
method1_gov_rdd6 = method1_gov_rdd5.map(lambda x: (x[0][0],(x[0][1],x[1])))
method1_gov_rdd6.take(5)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  (('agreement',), 1.0)),
 ('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  (('state',), 1.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  (('laws', 'regulations', 'license', 'conditions'), 16.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  (('canadian', 'radiotelevision'), 4.0)),
 ('FulucaiProductionsLtd_20131223_10-Q_EX-10.9_8368347_EX-10.9_Content License Agreement.pdf1',
  (('parties', 'hereto', 'shall'), 9.0))]

In [18]:
# using groupByKey() to group the candidate phrases with the same filename
method1_gov_phrase_score = method1_gov_rdd6.groupByKey().map(lambda x: (x[0], list(x[1])))
method1_gov_phrase_score.take(1)

[('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf1',
  [(('executed',), 1.0),
   (('state',), 2.0),
   (('agreement',), 1.0),
   (('performance', 'shall'), 4.0),
   (('laws',), 1.0),
   (('construed',), 1.0),
   (('enforced',), 1.0),
   (('los', 'angeles', 'county'), 9.0),
   (('california',), 2.0),
   (('interpretation', 'validity'), 4.0),
   (('delivered',), 1.0),
   (('accordance',), 1.0)])]

### 5-The edf, rdf and ess calculation for each candidate phrase

In [19]:
# generate the format like(candidate_phrase, ratio_score, edf_score, rdf_score)
# the first tuple in the entire tuple refers to one word of each document(clause), 
# the second is the list of the word’s candidate phrase
# the third number is 1 which represents the frequency of each single word for the candidate phrase
# and the fourth number means the degree of single word(frequency+ co-occurrence).

def method1_candidate_phrase(record):
  ls = record[1]
  ls.sort(key = lambda x:x[1], reverse = True)
  res = []
  for i in range(len(ls)):
    if i <= 3:
      x = (ls[i][0], ls[i][1], 1, 1)
    else:
      x = (ls[i][0], ls[i][1], 0, 1)
    res.append(x)                

  return  (record[0], res)


# (candidate_phrase, ratio_score, edf_score, rdf_score)
method1_gov_rdd8 = method1_gov_phrase_score.map(method1_candidate_phrase)
method1_gov_rdd8.take(1)

[('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf1',
  [(('los', 'angeles', 'county'), 9.0, 1, 1),
   (('performance', 'shall'), 4.0, 1, 1),
   (('interpretation', 'validity'), 4.0, 1, 1),
   (('state',), 2.0, 1, 1),
   (('california',), 2.0, 0, 1),
   (('executed',), 1.0, 0, 1),
   (('agreement',), 1.0, 0, 1),
   (('laws',), 1.0, 0, 1),
   (('construed',), 1.0, 0, 1),
   (('enforced',), 1.0, 0, 1),
   (('delivered',), 1.0, 0, 1),
   (('accordance',), 1.0, 0, 1)])]

#### 5.1. Edf calculation and keywords identification

In [20]:
def extract_edf(record):
  ls = record[1]
  res = []
  for word in ls:
    x = (word[0], word[2])
    res.append(x)
  return res

# filter the second element of the tuple being 0, because 0 means it is not the keyword
method1_gov_keyword_edf = method1_gov_rdd8.flatMap(extract_edf).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] != 0).sortBy(lambda x: x[1], ascending = False)
method1_gov_keyword_edf.take(10)

[(('agreement', 'shall'), 216),
 (('laws',), 126),
 (('governed',), 54),
 (('state',), 53),
 (('accordance',), 40),
 (('construed',), 36),
 (('agreement',), 33),
 (('law', 'principles'), 26),
 (('new', 'york'), 25),
 (('laws', 'principles'), 23)]

#### 5.2. Rdf calculation

In [21]:
def extract_rdf(record):
  ls = record[1]
  res = []
  for word in ls:
    x = (word[0], word[3])
    res.append(x)
  return res

method1_gov_keyword_rdf = method1_gov_rdd8.flatMap(extract_rdf).reduceByKey(lambda a,b: a+b).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_gov_keyword_rdf.take(10)

[(('laws',), 387),
 (('governed',), 358),
 (('state',), 329),
 (('accordance',), 286),
 (('construed',), 251),
 (('agreement', 'shall'), 250),
 (('agreement',), 168),
 (('conflict',), 112),
 (('conflicts',), 107),
 (('interpreted',), 50)]

#### 5.3. Ess calculation and the result

In [22]:
# ess = edf/rdf * edf
# gerenrate the tuple like (candidate phrase, (edf, rdf, ess))
method1_gov_ess = method1_gov_keyword_edf.join(method1_gov_keyword_rdf).map(lambda x: (x[0], (x[1][0], x[1][1] ,x[1][0]*x[1][0]/x[1][1]))).sortBy(lambda x: x[1][2], ascending = False)

## Filter the candidate phrases whose number of words is greater than 4
## Convert the tuple into string
method1_gov_final_ess = method1_gov_ess.filter(lambda x: len(x[0]) <= 4).map(lambda x: (' '.join(x[0]), x[1]))
method1_gov_final_ess.take(20)

[('agreement shall', (216, 250, 186.624)),
 ('laws', (126, 387, 41.02325581395349)),
 ('new york', (25, 28, 22.321428571428573)),
 ('new york without regard', (20, 20, 20.0)),
 ('law principles', (26, 36, 18.77777777777778)),
 ('laws principles', (23, 31, 17.06451612903226)),
 ('law rules', (16, 19, 13.473684210526315)),
 ('law provisions', (16, 19, 13.473684210526315)),
 ('substantive laws', (16, 21, 12.19047619047619)),
 ('new york applicable', (12, 12, 12.0)),
 ('california without regard', (12, 12, 12.0)),
 ('delaware without regard', (12, 12, 12.0)),
 ('laws provisions', (13, 17, 9.941176470588236)),
 ('performed entirely within', (9, 9, 9.0)),
 ('parties hereto shall', (9, 9, 9.0)),
 ('new york without reference', (9, 9, 9.0)),
 ('state', (53, 329, 8.537993920972644)),
 ('governed', (54, 358, 8.145251396648044)),
 ('laws principles thereof', (7, 7, 7.0)),
 ('validity construction', (7, 7, 7.0))]

## Change of Control

###  1-Map the original RDD to the tuple like [(filename1, clause1),(filename2, clause1)]

In [23]:
change_anti = spark.read.csv('/content/drive/MyDrive/comp5349/Anti_assignment_CIC_g3.csv', header = True).rdd
change_anti.take(5)

[Row(Filename='CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf', Change of Control=None, Anti-assignment='MA may not assign, sell, lease or otherwise transfer in whole or in party any of the rights granted pursuant to this Agreement without prior written approval of Company. (Page 12)'),
 Row(Filename='EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf', Change of Control=None, Anti-assignment='This Agreement may not be assigned, sold or transferred without the prior written consent of the other party. (Page 9)  Notwithstanding the foregoing, Rogers may, without consent, assign its rights and obligations under this Agreement in whole or in part to: (i) a person that directly or indirectly controls, is controlled by or is under common control with Rogers; or (ii) a purchaser of all or substantially all of the assets used in connection with the ROD Service. A change of control of Rogers shall not be consi

In [24]:
def row_to_tuple_for_control (row):
  row_dict = row.asDict()
  return (row_dict["Filename"], row_dict["Change of Control"])    

change_of_control = change_anti.map(row_to_tuple_for_control)

In [25]:
# map to this format [(filename1, clause1), (filename2, clause2)]
# Uing the function of method1_extract defined in the previous section
method1_change_rdd1 = change_of_control.filter(lambda x: x[1] != None).flatMap(method1_extract).map(lambda x: (x[0],x[1].strip())) #  filter None values and  spaces
method1_change_rdd1.take(10)

[('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
  "for purposes of the preceding sentence, and without limiting its generality, any merger, consolidation or reorganization involving licensee (regardless of whether licensee is a surviving or disappearing entity) will be deemed to be a transfer of rights, obligations or performance under this agreement for which licensor's prior written consent is required."),
 ('DeltathreeInc_19991102_S-1A_EX-10.19_6227850_EX-10.19_Co-Branding Agreement_ Service Agreement.pdf1',
  '"the term of this agreement shall be effective as of the date first stated above and shall continue for a term of three (3) years, unless terminated earlier in accordance with the provisions of this agreement (the ""term""); provided'),
 ('EdietsComInc_20001030_10QSB_EX-10.4_2606646_EX-10.4_Co-Branding Agreement.pdf1',
  '"for purposes of this agreement, ""change in control"" means a merger or consolidation of the party with'),
 ('Muscleph

### 2-candidate Phrase Identification

removing stopwords and punctuation, then generating candidate phrases.

In [26]:
# map to the format like (filename, list of candidate phrases)
method1_change_rdd2 = method1_change_rdd1.map(method1_preprocessing).filter(lambda x: x[1] != ' ')
method1_change_rdd2.take(1)

[('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
  ['',
   ' purposes ',
   ' ',
   ' preceding sentence ',
   ' without limiting ',
   ' generality ',
   ' merger consolidation ',
   ' reorganization involving licensee regardless ',
   ' whether licensee ',
   ' ',
   ' surviving ',
   ' disappearing entity ',
   ' ',
   ' deemed ',
   ' ',
   ' ',
   ' transfer ',
   ' rights obligations ',
   ' performance ',
   ' ',
   ' agreement ',
   ' ',
   ' licensors prior written consent ',
   ' required'])]

In [27]:
# remove extra spaces
# Map the candidate phrases to the list format, and each word is split by comma for each candidate phrase of each document
method1_change_rdd_list = method1_change_rdd2.map(remove_space_list)
method1_change_rdd_list.take(1)

[('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
  [['purposes'],
   ['preceding', 'sentence'],
   ['without', 'limiting'],
   ['generality'],
   ['merger', 'consolidation'],
   ['reorganization', 'involving', 'licensee', 'regardless'],
   ['whether', 'licensee'],
   ['surviving'],
   ['disappearing', 'entity'],
   ['deemed'],
   ['transfer'],
   ['rights', 'obligations'],
   ['performance'],
   ['agreement'],
   ['licensors', 'prior', 'written', 'consent'],
   ['required']])]

### 3- Word Score Calculation

In [28]:
# Generate the tuple like ((filename, single_word), candidate phrase,  fre(w), deg(w))) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
method1_change_rdd2 = method1_change_rdd_list.flatMap(ls_fre_deg)
method1_change_rdd2.take(3)

[(('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'purposes'),
  ['purposes'],
  1,
  1),
 (('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'preceding'),
  ['preceding', 'sentence'],
  1,
  2),
 (('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'sentence'),
  ['preceding', 'sentence'],
  1,
  2)]

In [29]:
# This RDD is for joining later
method1_change_rdd = method1_change_rdd2.map(lambda x: (x[0], x[1]))
method1_change_rdd.take(3)

[(('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'purposes'),
  ['purposes']),
 (('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'preceding'),
  ['preceding', 'sentence']),
 (('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
   'sentence'),
  ['preceding', 'sentence'])]

#### 3.1.Fre(w) calculation for each word

In [30]:
method1_change_fre = method1_change_rdd2.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_change_fre.take(5)

[(('ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.pdf2',
   'party'),
  11),
 (('ConformisInc_20191101_10-Q_EX-10.6_11861402_EX-10.6_Development Agreement.pdf1',
   'party'),
  10),
 (('HarpoonTherapeuticsInc_20200312_10-K_EX-10.18_12051356_EX-10.18_Development Agreement.PDF1',
   'section'),
  10),
 (('PhoenixNewMediaLtd_20110421_F-1_EX-10.17_6958322_EX-10.17_Content License Agreement.pdf1',
   'party'),
  10),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF1', 'interest'), 10)]

#### 3.2.Deg(w) calculation for each word

In [31]:
method1_change_deg = method1_change_rdd2.map(lambda x: (x[0], x[3])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_change_deg.take(5)

[(('HarpoonTherapeuticsInc_20200312_10-K_EX-10.18_12051356_EX-10.18_Development Agreement.PDF1',
   'section'),
  36),
 (('PhoenixNewMediaLtd_20110421_F-1_EX-10.17_6958322_EX-10.17_Content License Agreement.pdf1',
   'party'),
  32),
 (('ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.pdf2',
   'party'),
  29),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'written'),
  28),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'notice'),
  28)]

#### 3.3.Ratio of fre(w) and deg(w) calculation for each word

In [32]:
# ratio = deg/fre
method1_change_score = method1_change_fre.join(method1_change_deg).map(lambda x: (x[0],x[1][1]/x[1][0])).sortBy(lambda x: x[1], ascending = False)
method1_change_score.take(5)

[(('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'yahoo'),
  20.0),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'lycosterra'),
  20.0),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'nbc'),
  20.0),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'cmgi'),
  20.0),
 (('EbixInc_20010515_10-Q_EX-10.3_4049767_EX-10.3_Co-Branding Agreement.pdf1',
   'disney'),
  20.0)]

### 4-Candidate Phrase Score Calculation

In [33]:
# join the RDD lile (word,candidate phrase) and ((word, score) RDD
# using map and reduceByKey operations to generate like the following 
method1_change_rdd4 = method1_change_rdd.join(method1_change_score).map(lambda x: (x[0][0],(tuple(x[1][0]),x[1][1])))
method1_change_rdd5 = method1_change_rdd4.map(lambda x: ((x[0], x[1][0]),x[1][1])).reduceByKey(lambda a,b: a+b)
method1_change_rdd6 = method1_change_rdd5.map(lambda x: (x[0][0],(x[0][1],x[1])))
method1_change_rdd6.take(5)

[('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
  (('reorganization', 'involving', 'licensee', 'regardless'), 15.0)),
 ('GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10.1_Content License Agreement.pdf1',
  (('surviving',), 1.0)),
 ('DeltathreeInc_19991102_S-1A_EX-10.19_6227850_EX-10.19_Co-Branding Agreement_ Service Agreement.pdf1',
  (('three', '3', 'years', 'unless', 'terminated', 'earlier'), 36.0)),
 ('EdietsComInc_20001030_10QSB_EX-10.4_2606646_EX-10.4_Co-Branding Agreement.pdf1',
  (('agreement', 'change'), 4.0)),
 ('EdietsComInc_20001030_10QSB_EX-10.4_2606646_EX-10.4_Co-Branding Agreement.pdf1',
  (('party',), 1.0))]

In [34]:
# using groupByKey() to group the candidate phrases with the same filename
method1_change_phrase_score = method1_change_rdd6.groupByKey().map(lambda x: (x[0], list(x[1])))
method1_change_phrase_score.take(2)

[('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf1',
  [(('rights', 'andor', 'obligations', 'incurred', 'pursuant'), 25.0),
   (('agreement', 'without'), 4.0),
   (('party',), 2.5),
   (('otherwise', 'transfer'), 4.0),
   (('operation',), 1.0),
   (('law', 'assign'), 4.0),
   (('neither', 'party', 'shall', 'voluntarily'), 14.5),
   (('terms',), 1.0),
   (('prior', 'written', 'consent'), 9.0)]),
 ('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf2',
  [(('foregoing',), 1.0),
   (('agreement', 'may'), 3.5),
   (('assigned', 'without'), 4.0),
   (('partiesõ', 'consent'), 4.0),
   (('control', 'transaction'), 5.0),
   (('musclepharm',), 3.75),
   (('musclepharm', 'shall'), 3.75),
   (('change',), 2.0),
   (('control', 'transaction', 'provided'), 8.0),
   (('shall', 'specifically', 'assume'), 8.5),
   (('financial', 'resources', 'substantially', 'similar'), 16.0),
   (('acquirer',), 1.0),
   (('agreement',), 1.5),
   (('w

### 5-The edf, rdf and ess calculation for each candidate phrase


In [35]:
# generate the format like(candidate_phrase, ratio_score, edf_score, rdf_score)
# the first tuple in the entire tuple refers to one word of each document(clause), 
# the second is the list of the word’s candidate phrase
# the third number is 1 which represents the frequency of each single word for the candidate phrase
# and the fourth number means the degree of single word(frequency+ co-occurrence).

method1_change_rdd8 = method1_change_phrase_score.map(method1_candidate_phrase)
method1_change_rdd8.take(2)

[('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf1',
  [(('rights', 'andor', 'obligations', 'incurred', 'pursuant'), 25.0, 1, 1),
   (('neither', 'party', 'shall', 'voluntarily'), 14.5, 1, 1),
   (('prior', 'written', 'consent'), 9.0, 1, 1),
   (('agreement', 'without'), 4.0, 1, 1),
   (('otherwise', 'transfer'), 4.0, 0, 1),
   (('law', 'assign'), 4.0, 0, 1),
   (('party',), 2.5, 0, 1),
   (('operation',), 1.0, 0, 1),
   (('terms',), 1.0, 0, 1)]),
 ('MusclepharmCorp_20170208_10-KA_EX-10.38_9893581_EX-10.38_Co-Branding Agreement.pdf2',
  [(('financial', 'resources', 'substantially', 'similar'), 16.0, 1, 1),
   (('shall', 'specifically', 'assume'), 8.5, 1, 1),
   (('control', 'transaction', 'provided'), 8.0, 1, 1),
   (('control', 'transaction'), 5.0, 1, 1),
   (('assigned', 'without'), 4.0, 0, 1),
   (('partiesõ', 'consent'), 4.0, 0, 1),
   (('writing', 'prior'), 4.0, 0, 1),
   (('musclepharm',), 3.75, 0, 1),
   (('musclepharm', 'shall'), 3.75, 0, 1),

#### 5.1. Edf calculation and keywords identification

In [36]:
method1_change_edf = method1_change_rdd8.flatMap(extract_edf).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] != 0).sortBy(lambda x: x[1], ascending = False)
method1_change_edf.take(20)

[(('prior', 'written', 'consent'), 11),
 (('party',), 7),
 (('control',), 6),
 (('change',), 5),
 (('agreement', 'may'), 5),
 (('effective', 'date'), 4),
 (('ownership', 'transfer'), 4),
 (('written', 'notice'), 4),
 (('would', 'result'), 3),
 (('control', 'event'), 3),
 (('agreement', 'upon', 'written', 'notice'), 3),
 (('agreement',), 3),
 (('excitehome', 'named', 'competitor'), 3),
 (('agreement', 'shall', 'terminate'), 3),
 (('terminated', 'upon'), 3),
 (('third', 'party'), 3),
 (('purposes',), 3),
 (('control', 'shall'), 3),
 (('licensee',), 3),
 (('licensor', 'may', 'terminate'), 3)]

#### 5.2. Rdf calculation

In [37]:
method1_change_rdf = method1_change_rdd8.flatMap(extract_rdf).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_change_rdf.take(20)

[(('change',), 104),
 (('agreement',), 94),
 (('control',), 83),
 (('event',), 43),
 (('party',), 41),
 (('substantially',), 29),
 (('transfer',), 27),
 (('rights',), 27),
 (('terminate',), 26),
 (('sale',), 25),
 (('assets',), 25),
 (('right',), 24),
 (('assignment',), 22),
 (('connection',), 19),
 (('merger',), 19),
 (('obligations',), 17),
 (('operation',), 17),
 (('prior', 'written', 'consent'), 16),
 (('writing',), 16),
 (('termination',), 16)]

#### 5.3. Ess calculation and the result

In [38]:
# ess = edf/rdf * edf
# gerenrate the tuple like (candidate phrase, (edf, rdf, ess))
method1_change_ess = method1_change_edf.join(method1_change_rdf).map(lambda x: (x[0], (x[1][0], x[1][1] ,x[1][0]*x[1][0]/x[1][1]))).sortBy(lambda x: x[1][2], ascending = False)

## Filter the candidate phrases whose number of words is greater than 4
## Convert the tuple into string
method1_change_final_ess = method1_change_ess.filter(lambda x: len(x[0]) <= 4).map(lambda x: (' '.join(x[0]), x[1]))
method1_change_final_ess.take(20)

[('prior written consent', (11, 16, 7.5625)),
 ('ownership transfer', (4, 4, 4.0)),
 ('agreement upon written notice', (3, 3, 3.0)),
 ('terminated upon', (3, 3, 3.0)),
 ('control shall', (3, 3, 3.0)),
 ('excitehome named competitor', (3, 3, 3.0)),
 ('licensor may terminate', (3, 3, 3.0)),
 ('would result', (3, 4, 2.25)),
 ('agreement shall terminate', (3, 4, 2.25)),
 ('made third party beneficiaries', (2, 2, 2.0)),
 ('party may terminate', (2, 2, 2.0)),
 ('providing written notice', (2, 2, 2.0)),
 ('control means', (2, 2, 2.0)),
 ('control shall occur', (2, 2, 2.0)),
 ('section 16av due', (2, 2, 2.0)),
 ('shall provide written notice', (2, 2, 2.0)),
 ('authority granted hereunder shall', (2, 2, 2.0)),
 ('agreement shall immediately cease', (2, 2, 2.0)),
 ('sixty 60 days', (2, 2, 2.0)),
 ('providing prior written notice', (2, 2, 2.0))]

## Anti-assignment

### 1-Map the original RDD to the tuple like [(filename1, clause1),(filename2, clause1)]

In [39]:
def row_to_tuple_for_anti(row):
  row_dict = row.asDict()
  return (row_dict["Filename"], row_dict["Anti-assignment"])       

anti_assignment = change_anti.map(row_to_tuple_for_anti)

In [40]:
# map to this format [(filename1, clause1), (filename2, clause2)]
# Uing the function of method1_extract defined in the previous section
method1_anti_rdd1 = anti_assignment.filter(lambda x: x[1] != None).flatMap(method1_extract).map(lambda x: (x[0],x[1].strip())) #  filter None values and  spaces
method1_anti_rdd1.take(10)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  'ma may not assign, sell, lease or otherwise transfer in whole or in party any of the rights granted pursuant to this agreement without prior written approval of company.'),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf1',
  'this agreement may not be assigned, sold or transferred without the prior written consent of the other party.'),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  'notwithstanding the foregoing, rogers may, without consent, assign its rights and obligations under this agreement in whole or in part to: (i) a person that directly or indirectly controls, is controlled by or is under common control with rogers; or (ii) a purchaser of all or substantially all of the assets used in connection with the rod service. a change of control of rogers shall not be considered 

### 2-candidate Phrase Identification

removing stopwords and punctuation, then generating candidate phrases.

In [41]:
# map to the format like (filename, list of candidate phrases)
method1_anti_rdd2 = method1_anti_rdd1.map(method1_preprocessing).filter(lambda x: x[1] != ' ')
method1_anti_rdd2.take(2)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  ['',
   ' may ',
   ' assign sell lease ',
   ' otherwise transfer ',
   ' whole ',
   ' ',
   ' party ',
   ' ',
   ' ',
   ' rights granted pursuant ',
   ' ',
   ' agreement without prior written approval ',
   ' company']),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf1',
  ['',
   ' agreement may ',
   ' ',
   ' assigned sold ',
   ' transferred without ',
   ' prior written consent ',
   ' ',
   ' ',
   ' party'])]

In [42]:
# remove extra spaces
# Map the candidate phrases to the list format, and each word is split by comma for each candidate phrase of each document
method1_anti_rdd_list = method1_anti_rdd2.map(remove_space_list)
method1_anti_rdd_list.take(2)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  [['may'],
   ['assign', 'sell', 'lease'],
   ['otherwise', 'transfer'],
   ['whole'],
   ['party'],
   ['rights', 'granted', 'pursuant'],
   ['agreement', 'without', 'prior', 'written', 'approval'],
   ['company']]),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf1',
  [['agreement', 'may'],
   ['assigned', 'sold'],
   ['transferred', 'without'],
   ['prior', 'written', 'consent'],
   ['party']])]

### 3- Word Score Calculation

In [43]:
# Generate the tuple like ((filename, single_word), candidate phrase,  fre(w), deg(w))) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
method1_anti_rdd2 = method1_anti_rdd_list.flatMap(ls_fre_deg)
method1_anti_rdd2.take(3)

[(('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'may'),
  ['may'],
  1,
  1),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'assign'),
  ['assign', 'sell', 'lease'],
  1,
  3),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'sell'),
  ['assign', 'sell', 'lease'],
  1,
  3)]

In [44]:
# This RDD is for joining later
method1_anti_rdd = method1_anti_rdd2.map(lambda x: (x[0], x[1]))
method1_anti_rdd.take(3)

[(('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'may'),
  ['may']),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'assign'),
  ['assign', 'sell', 'lease']),
 (('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
   'sell'),
  ['assign', 'sell', 'lease'])]

#### 3.1.Fre(w) calculation for each word

In [45]:
method1_anti_fre = method1_anti_rdd2.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_anti_fre.take(5)

[(('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'franchise'), 14),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'new'), 14),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'must'), 13),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'us'), 13),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'proposed'), 12)]

#### 3.2.Deg(w) calculation for each word

In [46]:
method1_anti_deg = method1_anti_rdd2.map(lambda x: (x[0], x[3])).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_anti_deg.take(5)

[(('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'new'), 53),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'proposed'), 47),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'owner'), 39),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'must'), 38),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'franchise'), 33)]

#### 3.3.Ratio of fre(w) and deg(w) calculation for each word

In [47]:
method1_anti_score = method1_anti_fre.join(method1_anti_deg).map(lambda x: (x[0],x[1][1]/x[1][0]))
method1_anti_score.take(5)

[(('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'franchise'),
  2.357142857142857),
 (('ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.pdf1',
   'party'),
  2.6363636363636362),
 (('JOINTCORP_09_19_2014-EX-10.15-FRANCHISE AGREEMENT.PDF3', 'transfer'),
  1.6363636363636365),
 (('RevolutionMedicinesInc_20200117_S-1_EX-10.1_11948417_EX-10.1_Development Agreement.pdf4',
   'family'),
  2.8),
 (('UpjohnInc_20200121_10-12G_EX-2.6_11948692_EX-2.6_Manufacturing Agreement_ Supply Agreement.pdf1',
   'facility'),
  1.875)]

### 4-Candidate Phrase Score Calculation

In [48]:
method1_anti_rdd4 = method1_anti_rdd.join(method1_anti_score).map(lambda x: (x[0][0],(tuple(x[1][0]),x[1][1])))
method1_anti_rdd5 = method1_anti_rdd4.map(lambda x: ((x[0], x[1][0]),x[1][1])).reduceByKey(lambda a,b: a+b)
method1_anti_rdd6 = method1_anti_rdd5.map(lambda x: (x[0][0],(x[0][1],x[1])))
method1_anti_rdd6.take(5)

[('CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf1',
  (('agreement', 'without', 'prior', 'written', 'approval'), 25.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf1',
  (('assigned', 'sold'), 4.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  (('whole',), 1.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  (('part',), 1.0)),
 ('EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B(01)_525118_EX-10.B(01)_Content License Agreement.pdf2',
  (('person',), 1.0))]

In [49]:
# using groupByKey() to group the candidate phrases with the same filename
method1_anti_phrase_score = method1_anti_rdd6.groupByKey().map(lambda x: (x[0], list(x[1])))
method1_anti_phrase_score.take(1)

[('IdeanomicsInc_20160330_10-K_EX-10.26_9512211_EX-10.26_Content License Agreement.pdf1',
  [(('assign',), 1.0),
   (('right',), 1.0),
   (('whole',), 1.0),
   (('licenseeõs', 'business'), 4.0),
   (('part',), 1.0),
   (('prior', 'written', 'notice'), 9.0),
   (('licensor',), 1.0),
   (('agreement',), 1.0),
   (('subject',), 1.0),
   (('licensee', 'shall'), 4.0),
   (('sublicense',), 1.0),
   (('third', 'parties', 'exhibiting'), 9.0),
   (('ordinary', 'course'), 4.0),
   (('section', '2ai', 'license', 'grant'), 16.0),
   (('rights', 'granted'), 4.0),
   (('applicable', 'limitations'), 4.0),
   (('titles',), 1.0)])]

### 5-The edf, rdf and ess calculation for each candidate phrase

In [55]:
# generate the format like(candidate_phrase, ratio_score, edf_score, rdf_score)
# the first tuple in the entire tuple refers to one word of each document(clause), 
# the second is the list of the word’s candidate phrase
# the third number is 1 which represents the frequency of each single word for the candidate phrase
# and the fourth number means the degree of single word(frequency+ co-occurrence).
method1_anti_rdd8 = method1_anti_phrase_score.map(method1_candidate_phrase)
method1_anti_rdd8.take(1)

[('IdeanomicsInc_20160330_10-K_EX-10.26_9512211_EX-10.26_Content License Agreement.pdf1',
  [(('section', '2ai', 'license', 'grant'), 16.0, 1, 1),
   (('prior', 'written', 'notice'), 9.0, 1, 1),
   (('third', 'parties', 'exhibiting'), 9.0, 1, 1),
   (('licenseeõs', 'business'), 4.0, 1, 1),
   (('licensee', 'shall'), 4.0, 0, 1),
   (('ordinary', 'course'), 4.0, 0, 1),
   (('rights', 'granted'), 4.0, 0, 1),
   (('applicable', 'limitations'), 4.0, 0, 1),
   (('assign',), 1.0, 0, 1),
   (('right',), 1.0, 0, 1),
   (('whole',), 1.0, 0, 1),
   (('part',), 1.0, 0, 1),
   (('licensor',), 1.0, 0, 1),
   (('agreement',), 1.0, 0, 1),
   (('subject',), 1.0, 0, 1),
   (('sublicense',), 1.0, 0, 1),
   (('titles',), 1.0, 0, 1)])]

#### 5.1. Edf calculation and keywords identification

In [56]:
method1_anti_edf = method1_anti_rdd8.flatMap(extract_edf).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] != 0).sortBy(lambda x: x[1], ascending = False)
method1_anti_edf.take(20)

[(('prior', 'written', 'consent'), 206),
 (('neither', 'party', 'may', 'assign'), 43),
 (('agreement', 'without'), 40),
 (('agreement',), 37),
 (('void',), 36),
 (('party',), 33),
 (('either', 'party', 'without'), 30),
 (('agreement', 'may'), 26),
 (('unreasonably', 'withheld'), 23),
 (('either', 'party', 'may', 'assign'), 22),
 (('attempted', 'assignment'), 20),
 (('third', 'party'), 20),
 (('agreement', 'shall'), 20),
 (('consent', 'shall'), 19),
 (('assignment',), 19),
 (('obligations', 'hereunder', 'without'), 16),
 (('third', 'party', 'without'), 16),
 (('unreasonably', 'withheld', 'conditioned'), 15),
 (('party', 'may', 'assign'), 13),
 (('null',), 13)]

#### 5.2. Rdf calculation

In [57]:
method1_anti_rdf = method1_anti_rdd8.flatMap(extract_rdf).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
method1_anti_rdf.take(20)

[(('agreement',), 303),
 (('prior', 'written', 'consent'), 242),
 (('rights',), 221),
 (('party',), 177),
 (('obligations',), 135),
 (('assign',), 128),
 (('assignment',), 100),
 (('void',), 99),
 (('transfer',), 98),
 (('assigned',), 92),
 (('substantially',), 86),
 (('whole',), 81),
 (('agreement', 'without'), 80),
 (('right',), 74),
 (('assets',), 72),
 (('part',), 70),
 (('consent',), 69),
 (('operation',), 66),
 (('shall',), 55),
 (('sale',), 55)]

#### 5.3. Ess calculation and the result

In [58]:
# ess = edf * edf/rdf 
# gerenrate the tuple like (candidate phrase, (edf, rdf, ess))
method1_anti_ess = method1_anti_edf.join(method1_anti_rdf).map(lambda x: (x[0], (x[1][0], x[1][1] ,x[1][0]*x[1][0]/x[1][1]))).sortBy(lambda x: x[1][2], ascending = False)

## Filter the candidate phrases whose number of words is greater than 4
## Convert the tuple into string
method1_anti_final_ess = method1_anti_ess.filter(lambda x: len(x[0]) <= 4).map(lambda x: (' '.join(x[0]), x[1]))
method1_anti_final_ess.take(20)

[('prior written consent', (206, 242, 175.35537190082644)),
 ('neither party may assign', (43, 44, 42.02272727272727)),
 ('either party without', (30, 40, 22.5)),
 ('either party may assign', (22, 23, 21.043478260869566)),
 ('agreement without', (40, 80, 20.0)),
 ('unreasonably withheld conditioned', (15, 15, 15.0)),
 ('attempted assignment', (20, 27, 14.814814814814815)),
 ('agreement may', (26, 49, 13.795918367346939)),
 ('third party without', (16, 19, 13.473684210526315)),
 ('void', (36, 99, 13.090909090909092)),
 ('neither party shall assign', (12, 12, 12.0)),
 ('express written consent', (12, 12, 12.0)),
 ('obligations hereunder without', (16, 25, 10.24)),
 ('prior written approval', (10, 10, 10.0)),
 ('party may assign', (13, 17, 9.941176470588236)),
 ('unreasonably withheld', (23, 54, 9.796296296296296)),
 ('agreement shall', (20, 44, 9.090909090909092)),
 ('neither party shall', (10, 11, 9.090909090909092)),
 ('express prior written consent', (9, 9, 9.0)),
 ('consent shall', (

# Method 2

## Governing_law

### 1-Candidate Phrase Identification

In [59]:
def gov_row_to_tuple (row):
  row_dict = row.asDict()
  return row_dict["Governing Law"]

gov_rdd = Governing_law.map(gov_row_to_tuple)

In [60]:
# removing pages and covert all into lower case
def preprocessing(record):
  doc = re.sub(r'\([Pp]age.*?\)', "", record) # removing pages
  doc = doc.lower() # convert lowercase
  return doc

# filter nan and delete spaces
gov_rdd1 = gov_rdd.map(preprocessing).filter(lambda x: x != 'nan').map(lambda x: x.strip())
gov_rdd1.take(5)

['this agreement is accepted by company in the state of nevada and shall be governed by and construed in accordance with the laws thereof, which laws shall prevail in the event of any conflict.',
 'this agreement shall be governed by laws of the province of ontario and the federal laws of canada applicable therein.   this agreement is subject to all laws, regulations, license conditions and decisions of the canadian radio-television and telecommunications commission (‚äúcrtc‚äù) municipal, provincial and federal governments or other authorities which are applicable to rogers and/or licensor, and which are now in force or hereafter adopted (‚äúapplicable law‚äù).',
 'all questions with respect to the construction of this agreement, and the rights and liabilities of the parties hereto, shall be governed by the laws of the state of florida.',
 'this agreement shall be governed by and construed in accordance with the internal laws of the state of nevada without giving effect to any choice 

In [61]:
def ExtractContentWord(record):
  docs = ''
  for word in record:
    if word in punctuation:  # convert punctuation into "|"
      docs += "|"
    else:
      docs += word

  docs = re.sub(r'[^\w\s]', '|', docs)  # remove other punctuation

  doc_word = docs.split(" ")             
  for word in stopwords:             # convert stopwords into "|"
      doc_word = [ "|" if word == w else w for w in doc_word]   
     
 
  sentence = " ".join(doc_word)   
  sentence = sentence.split("|")  
  return sentence

# remove empty values and spacess
gov_list_candidate = gov_rdd1.flatMap(ExtractContentWord).filter(lambda x: x != ' ').map(lambda x: x.strip()).filter(lambda x: x!= '') 
gov_list_candidate.take(10)

['agreement',
 'accepted',
 'company',
 'state',
 'nevada',
 'shall',
 'governed',
 'construed',
 'accordance',
 'laws thereof']

In [62]:
# Convert the candidate phrase into list format like ['agreement']
def Tolist(record):
  try:
    ls = record.split(" ")
    return ls
  except:
    return [record]

# return (word, candidate phrases, fre(w), deg(w))
def ExtractEachWord(record):
  ls = []
  for i in record:
    ls.append((i, record, 1, len(record)))
  return ls


# Generate distinct candidate phrase and return (word, candidate_phrase) which is for joining with score of single word and using filter operation to remove the number of words greater tahn 4
gov_distinct = gov_list_candidate.distinct().map(Tolist).flatMap(ExtractEachWord).map(lambda x: (x[0], x[1])).filter(lambda x: len(x[1]) <= 4) 
gov_distinct.take(20)

[('agreement', ['agreement']),
 ('accepted', ['accepted']),
 ('company', ['company']),
 ('state', ['state']),
 ('nevada', ['nevada']),
 ('shall', ['shall']),
 ('governed', ['governed']),
 ('construed', ['construed']),
 ('accordance', ['accordance']),
 ('laws', ['laws', 'thereof']),
 ('thereof', ['laws', 'thereof']),
 ('laws', ['laws', 'shall', 'prevail']),
 ('shall', ['laws', 'shall', 'prevail']),
 ('prevail', ['laws', 'shall', 'prevail']),
 ('event', ['event']),
 ('conflict', ['conflict']),
 ('agreement', ['agreement', 'shall']),
 ('shall', ['agreement', 'shall']),
 ('laws', ['laws']),
 ('province', ['province'])]

### 2-Word Score Calculation

In [63]:
# Generate the tuple like (single word, candidate phrase,  fre(w), deg(w))) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
gov_list_score = gov_list_candidate.map(Tolist).flatMap(ExtractEachWord)
gov_list_score.take(10)

[('agreement', ['agreement'], 1, 1),
 ('accepted', ['accepted'], 1, 1),
 ('company', ['company'], 1, 1),
 ('state', ['state'], 1, 1),
 ('nevada', ['nevada'], 1, 1),
 ('shall', ['shall'], 1, 1),
 ('governed', ['governed'], 1, 1),
 ('construed', ['construed'], 1, 1),
 ('accordance', ['accordance'], 1, 1),
 ('laws', ['laws', 'thereof'], 1, 2)]

#### 2.1.Fre(w) calculation for each word

In [64]:
gov_fre = gov_list_score.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
gov_fre.take(10)

[('laws', 622),
 ('agreement', 478),
 ('state', 423),
 ('shall', 397),
 ('governed', 389),
 ('accordance', 295),
 ('construed', 279),
 ('without', 225),
 ('law', 213),
 ('principles', 145)]

#### 2.2.Deg(w) calculation for each word

In [65]:
gov_deg =  gov_list_score.map(lambda x:(x[0], x[3])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
gov_deg.take(10)

[('shall', 904),
 ('laws', 831),
 ('agreement', 806),
 ('without', 704),
 ('state', 471),
 ('governed', 451),
 ('law', 447),
 ('new', 422),
 ('york', 394),
 ('accordance', 342)]

#### 2.3.Ratio of fre(w) and deg(w) calculation for each word

In [66]:
def ratio(record):
  # ratio = deg/fre
  ratio = record[1][1]/record[1][0]
  return (record[0], ratio)

#join fre and deg, then calcuate the ratio
gov_ratio = gov_fre.join(gov_deg).map(ratio).sortBy(lambda x: x[1], ascending = False)
gov_ratio.take(10)

[('thereunder', 12.5),
 ('whole', 11.0),
 ('provider', 10.0),
 ('necessarily', 10.0),
 ('payment', 8.0),
 ('systems', 8.0),
 ('data', 8.0),
 ('control', 6.5),
 ('sections', 6.0),
 ('practices', 6.0)]

### 3-Candidate Keyword Score Calculation and Final Results

In [67]:
gov_join = gov_distinct.join(gov_ratio).map(lambda x: (x[1][0], x[1][1]))
gov_join.take(10)

[(['accepted'], 1.0),
 (['state'], 1.1134751773049645),
 (['new', 'york', 'state'], 1.1134751773049645),
 (['united', 'state'], 1.1134751773049645),
 (['different', 'state'], 1.1134751773049645),
 (['actual', 'state'], 1.1134751773049645),
 (['another', 'state'], 1.1134751773049645),
 (['state', 'applicable'], 1.1134751773049645),
 (['state', 'court'], 1.1134751773049645),
 (['state', 'courts', 'therein', 'shall'], 1.1134751773049645)]

In [68]:
def listTostr(record):
  record = (' '.join(record[0]), record[1])
  return record

gov_result = gov_join.map(listTostr).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
gov_result.take(20)

[('sections 1051 et seq', 18.4),
 ('lj 1050 et seq', 16.4),
 ('1 covering competition following', 15.75),
 ('laws principles thereunder', 15.677392172081161),
 ('met independently without reference', 14.203888888888889),
 ('either party herein initiate', 13.75),
 ('without giving effect to', 13.181520467836258),
 ('1980 united nations convention', 12.855821371610846),
 ('performed entirely within pennsylvania', 12.686609686609687),
 ('maryland without giving effect', 12.431520467836258),
 ('1051 et seq', 12.399999999999999),
 ('new york without reference', 12.269493491167902),
 ('intellectual property right applies', 12.238095238095237),
 ('new york without recourse', 12.194493491167902),
 ('pennsylvania without giving effect', 12.181520467836258),
 ('new york without regard', 12.02355332022773),
 ('california without giving effect', 11.998421876286962),
 ('agreement shall become valid', 11.963270554261564),
 ('massachusetts without giving effect', 11.895806182121971),
 ('parties heret

## Change of Control

### 1-Candidate Phrase Identification

In [69]:
def change_row_to_tuple (row):
  row_dict = row.asDict()
  return row_dict["Change of Control"]

change_rdd = change_anti.map(change_row_to_tuple)
change_rdd.take(10)

[None,
 None,
 "For purposes of the preceding sentence, and without limiting its generality, any merger, consolidation or reorganization involving Licensee (regardless of whether Licensee is a surviving or disappearing entity) will be deemed to be a transfer of rights, obligations or performance under this Agreement for which Licensor's prior written consent is required. (Page 15)",
 None,
 '"The term of this Agreement shall be effective as of the date first stated above and shall continue for a term of three (3) years, unless terminated earlier in accordance with the provisions of this Agreement (the ""Term""); provided',
 '"For purposes of this Agreement, ""Change in Control"" means a merger or consolidation of the party with',
 None,
 'Neither party shall voluntarily or by operation of law assign or otherwise transfer the rights and/or obligations incurred pursuant to the terms of this Agreement without the prior written consent of the other party. (Page 21)  Nothwithstanding the fo

In [70]:
# removing pages and covert all into lowercase
# filter None and delete space
change_rdd1 = change_rdd.filter(lambda x: x != None).map(preprocessing).map(lambda x: x.strip())
change_rdd1.take(5)

["for purposes of the preceding sentence, and without limiting its generality, any merger, consolidation or reorganization involving licensee (regardless of whether licensee is a surviving or disappearing entity) will be deemed to be a transfer of rights, obligations or performance under this agreement for which licensor's prior written consent is required.",
 '"the term of this agreement shall be effective as of the date first stated above and shall continue for a term of three (3) years, unless terminated earlier in accordance with the provisions of this agreement (the ""term""); provided',
 '"for purposes of this agreement, ""change in control"" means a merger or consolidation of the party with',
 'neither party shall voluntarily or by operation of law assign or otherwise transfer the rights and/or obligations incurred pursuant to the terms of this agreement without the prior written consent of the other party.   nothwithstanding the foregoing, this agreement may be assigned without

In [71]:
# Extract candidate phrases and  remove empty values and spacess
change_list_candidate = change_rdd1.flatMap(ExtractContentWord).filter(lambda x: x != ' ').map(lambda x: x.strip()).filter(lambda x: x!= '') 
change_list_candidate.take(10)

['purposes',
 'preceding sentence',
 'without limiting',
 'generality',
 'merger',
 'consolidation',
 'reorganization involving licensee',
 'regardless',
 'whether licensee',
 'surviving']

In [72]:
# Generate distinct candidate phrase and return (word, candidate_phrase) which is for joining with score of single word
# filtered the number of words for candidates is greater than 4
change_distinct = change_list_candidate.distinct().map(Tolist).flatMap(ExtractEachWord).map(lambda x: (x[0], x[1])).filter(lambda x: len(x[1]) <= 4) 
change_distinct.take(20)

[('purposes', ['purposes']),
 ('preceding', ['preceding', 'sentence']),
 ('sentence', ['preceding', 'sentence']),
 ('without', ['without', 'limiting']),
 ('limiting', ['without', 'limiting']),
 ('generality', ['generality']),
 ('merger', ['merger']),
 ('consolidation', ['consolidation']),
 ('reorganization', ['reorganization', 'involving', 'licensee']),
 ('involving', ['reorganization', 'involving', 'licensee']),
 ('licensee', ['reorganization', 'involving', 'licensee']),
 ('regardless', ['regardless']),
 ('whether', ['whether', 'licensee']),
 ('licensee', ['whether', 'licensee']),
 ('surviving', ['surviving']),
 ('disappearing', ['disappearing', 'entity']),
 ('entity', ['disappearing', 'entity']),
 ('deemed', ['deemed']),
 ('transfer', ['transfer']),
 ('rights', ['rights'])]

### 2-Word Score Calculation

In [73]:
# Generate the tuple like (single word, candidate phrase,  fre(w), deg(w)) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
change_list_score = change_list_candidate.map(Tolist).flatMap(ExtractEachWord)
change_list_score.take(10)

[('purposes', ['purposes'], 1, 1),
 ('preceding', ['preceding', 'sentence'], 1, 2),
 ('sentence', ['preceding', 'sentence'], 1, 2),
 ('without', ['without', 'limiting'], 1, 2),
 ('limiting', ['without', 'limiting'], 1, 2),
 ('generality', ['generality'], 1, 1),
 ('merger', ['merger'], 1, 1),
 ('consolidation', ['consolidation'], 1, 1),
 ('reorganization', ['reorganization', 'involving', 'licensee'], 1, 3),
 ('involving', ['reorganization', 'involving', 'licensee'], 1, 3)]

#### 2.1.Fre(w) calculation for each word

In [74]:
change_fre = change_list_score.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
change_fre.take(10)

[('agreement', 244),
 ('party', 203),
 ('change', 190),
 ('control', 180),
 ('shall', 168),
 ('may', 98),
 ('notice', 97),
 ('written', 92),
 ('terminate', 80),
 ('transfer', 79)]

#### 2.2.Deg(w) calculation for each word

In [75]:
change_deg =  change_list_score.map(lambda x:(x[0], x[3])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
change_deg.take(10)

[('shall', 515),
 ('party', 445),
 ('agreement', 422),
 ('written', 336),
 ('notice', 302),
 ('may', 284),
 ('control', 265),
 ('change', 219),
 ('upon', 191),
 ('terminate', 184)]

#### 2.3.Ratio of fre(w) and deg(w) calculation for each word

In [76]:
#join fre and deg, then calcuate the ratio
change_ratio = change_fre.join(change_deg).map(ratio).sortBy(lambda x: x[1], ascending = False)
change_ratio.take(10)

[('furnish', 25.0),
 ('designates', 12.0),
 ('jointly', 10.0),
 ('two', 10.0),
 ('pretzel', 9.666666666666666),
 ('wireless', 9.0),
 ('added', 9.0),
 ('re', 9.0),
 ('ncmõs', 7.0),
 ('enterprise', 7.0)]

### 3-Candidate Keyword Score Calculation and Final Results

In [77]:
change_join = change_distinct.join(change_ratio).map(lambda x: (x[1][0], x[1][1]))
change_join.take(10)

[(['merger'], 1.3508771929824561),
 (['merger', 'transaction'], 1.3508771929824561),
 (['reverse', 'triangular', 'merger'], 1.3508771929824561),
 (['reorganization', 'involving', 'licensee'], 1.6756756756756757),
 (['whether', 'licensee'], 1.6756756756756757),
 (['1', 'licensee'], 1.6756756756756757),
 (['manage', 'licensee'], 1.6756756756756757),
 (['licensee'], 1.6756756756756757),
 (['licensee', 'prior'], 1.6756756756756757),
 (['licensee', 'may'], 1.6756756756756757)]

In [78]:
# convert the list of candidate into string format
change_result = change_join.map(listTostr).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
change_result.take(20)

[('home named competitor', 14.920634920634921),
 ('five thousand dollars', 14.857142857142858),
 ('vs key leadership position', 14.5),
 ('providing ebix written notice', 14.015575974899148),
 ('the òroyalty offset periodó', 14.0),
 ('upon sending written notice', 13.948909308232482),
 ('reasonable detail based upon', 13.783333333333333),
 ('without thereby becoming liable', 13.671186440677966),
 ('party representing fifty percent', 13.438871473354233),
 ('authority granted hereunder shall', 13.394047619047619),
 ('providing prior written notice', 13.172438719997187),
 ('upon written prior notice', 13.10577205333052),
 ('admit additional general partners', 13.05),
 ('party shall remain liable', 13.032594417077176),
 ('janssenõs confidential information hereunder', 13.019047619047619),
 ('shall provide written notice', 12.97390930823248),
 ('golf instruction related products', 12.971428571428572),
 ('section 14 ð òterminationó', 12.889830508474576),
 ('by providing written notice', 12.84

## Anti-assignment

### 1- Generate the list of candidate phrases

In [79]:
def anti_row_to_tuple (row):
  row_dict = row.asDict()
  return row_dict["Anti-assignment"]

anti_rdd = change_anti.map(anti_row_to_tuple)
anti_rdd.take(10)

['MA may not assign, sell, lease or otherwise transfer in whole or in party any of the rights granted pursuant to this Agreement without prior written approval of Company. (Page 12)',
 'This Agreement may not be assigned, sold or transferred without the prior written consent of the other party. (Page 9)  Notwithstanding the foregoing, Rogers may, without consent, assign its rights and obligations under this Agreement in whole or in part to: (i) a person that directly or indirectly controls, is controlled by or is under common control with Rogers; or (ii) a purchaser of all or substantially all of the assets used in connection with the ROD Service. A change of control of Rogers shall not be considered an assignment of this Agreement. (Page 9)  Any purported assignment, sale, or transfer in contravention of this Section shall be null and void. (Page 9)',
 "Licensee shall not assign or otherwise transfer any of its rights, or delegate or otherwise transfer any of its obligations or perfor

In [80]:
# removing pages and covert all into lowercase
# filter None and delete space
anti_rdd1 = anti_rdd.filter(lambda x: x != None).map(preprocessing).map(lambda x: x.strip())
anti_rdd1.take(5)

['ma may not assign, sell, lease or otherwise transfer in whole or in party any of the rights granted pursuant to this agreement without prior written approval of company.',
 'this agreement may not be assigned, sold or transferred without the prior written consent of the other party.   notwithstanding the foregoing, rogers may, without consent, assign its rights and obligations under this agreement in whole or in part to: (i) a person that directly or indirectly controls, is controlled by or is under common control with rogers; or (ii) a purchaser of all or substantially all of the assets used in connection with the rod service. a change of control of rogers shall not be considered an assignment of this agreement.   any purported assignment, sale, or transfer in contravention of this section shall be null and void.',
 "licensee shall not assign or otherwise transfer any of its rights, or delegate or otherwise transfer any of its obligations or performance, under this agreement, in eac

In [81]:
# Extract candidate phrases and  remove empty values and spacess
anti_list_candidate = anti_rdd1.flatMap(ExtractContentWord).filter(lambda x: x != ' ').map(lambda x: x.strip()).filter(lambda x: x!= '') 
anti_list_candidate.take(10)

['may',
 'assign',
 'sell',
 'lease',
 'otherwise transfer',
 'whole',
 'party',
 'rights granted pursuant',
 'agreement without prior written approval',
 'company']

In [82]:
# Generate distinct candidate phrase and return (word, candidate_phrase) which is for joining with score of single word
# filtered the number of words for candidates is greater than 4
anti_distinct = anti_list_candidate.distinct().map(Tolist).flatMap(ExtractEachWord).map(lambda x: (x[0], x[1])).filter(lambda x: len(x[1]) <= 4) 
anti_distinct.take(10)

[('may', ['may']),
 ('assign', ['assign']),
 ('sell', ['sell']),
 ('lease', ['lease']),
 ('otherwise', ['otherwise', 'transfer']),
 ('transfer', ['otherwise', 'transfer']),
 ('whole', ['whole']),
 ('party', ['party']),
 ('rights', ['rights', 'granted', 'pursuant']),
 ('granted', ['rights', 'granted', 'pursuant'])]

### 2-Word Score Calculation

In [83]:
# Generate the tuple like (single word, candidate phrase,  fre(w), deg(w)) for each phrase of each document
# the third number of each tuple is 1 which means frequency of each word for the one candidate phrase 
# and the fourth number is the length of candidate phrases since deg(w) = fre(w) + co-occurrence.
# In detail, fre(w) is equal to 1, and co-occurrence equals the length(candidate_phrase)-1. In this way, deg(w) = length of candidate phrase.
anti_list_score = anti_list_candidate.map(Tolist).flatMap(ExtractEachWord)
anti_list_score.take(10)

[('may', ['may'], 1, 1),
 ('assign', ['assign'], 1, 1),
 ('sell', ['sell'], 1, 1),
 ('lease', ['lease'], 1, 1),
 ('otherwise', ['otherwise', 'transfer'], 1, 2),
 ('transfer', ['otherwise', 'transfer'], 1, 2),
 ('whole', ['whole'], 1, 1),
 ('party', ['party'], 1, 1),
 ('rights', ['rights', 'granted', 'pursuant'], 1, 3),
 ('granted', ['rights', 'granted', 'pursuant'], 1, 3)]

#### 2.1. Fre(w) calculation for each word

In [84]:
anti_fre = anti_list_score.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
anti_fre.take(10)

[('party', 837),
 ('agreement', 797),
 ('consent', 559),
 ('shall', 534),
 ('without', 475),
 ('written', 441),
 ('assign', 411),
 ('may', 398),
 ('rights', 369),
 ('prior', 368)]

#### 2.2. Deg(w) calculation for each word

In [85]:
anti_deg =  anti_list_score.map(lambda x:(x[0], x[3])).reduceByKey(lambda a,b: a+b).filter(lambda x: x[0] != '').sortBy(lambda x: x[1], ascending = False)
anti_deg.take(10)

[('party', 1951),
 ('consent', 1659),
 ('written', 1591),
 ('shall', 1384),
 ('prior', 1323),
 ('agreement', 1228),
 ('without', 1188),
 ('may', 1170),
 ('assign', 950),
 ('hereunder', 579)]

#### 2.3.Ratio of fre(w) and deg(w) calculation for each word

In [86]:
#join fre and deg, then calcuate the ratio
anti_ratio = anti_fre.join(anti_deg).map(ratio).sortBy(lambda x: x[1], ascending = False)
anti_ratio.take(10)

[('accruing', 24.0),
 ('adjudicated', 21.0),
 ('managing', 14.0),
 ('decide', 14.0),
 ('insolvent', 12.0),
 ('medianet', 10.0),
 ('technologies', 10.0),
 ('holds', 9.8),
 ('schoolpop', 9.0),
 ('miltenyi', 9.0)]

### 3-Candidate Keyword Score Calculation and Final Results

In [87]:
anti_join = anti_distinct.join(anti_ratio).map(lambda x: (x[1][0], x[1][1]))
anti_join.take(10)

[(['sell'], 1.6129032258064515),
 (['neither', 'party', 'shall', 'sell'], 1.6129032258064515),
 (['way', 'sell'], 1.6129032258064515),
 (['sell', 'franchises'], 1.6129032258064515),
 (['sell', 'unit', 'franchises'], 1.6129032258064515),
 (['indirectly', 'sell'], 1.6129032258064515),
 (['lease'], 1.2222222222222223),
 (['lease', 'agreement'], 1.2222222222222223),
 (['rights', 'granted', 'pursuant'], 2.7346938775510203),
 (['rights', 'granted'], 2.7346938775510203)]

In [88]:
# convert the list of candidate into string format
anti_result = anti_join.map(listTostr).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending = False)
anti_result.take(20)

[('distributor becomes insolvent', 20.1),
 ('transporterõs ferc gas tariff', 15.5),
 ('cause forty niners sc', 15.022222222222222),
 ('assigning party holds', 14.988086704215737),
 ('express prior written authorization', 14.94640818980881),
 ('s prior written approval', 14.875798244198865),
 ('restrictions set forth herein', 14.870471014492754),
 ('indirect loss thus caused', 14.444444444444445),
 ('s prior written consent', 14.434506977326205),
 ('prior express written approval', 14.355499098899719),
 ('express prior written approval', 14.355499098899719),
 ('forty niners sc without', 14.301052631578948),
 ('nfla prior written consent', 14.170618088437315),
 ('party shall remain liable', 14.122704146698347),
 ('third party either becomes', 14.055865477166922),
 ('reynolds group holdings limited', 13.958333333333332),
 ('express prior written consent', 13.914207832027056),
 ('prior express written consent', 13.914207832027056),
 ('expressly set forth herein', 13.90893255295429),
 ('ass